In [1]:
# import modules
from sklearn import datasets
import pandas as pd
import geopandas as gpd
import numpy as np
import math
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from random import randint
import os

In [5]:
boston_df = pd.read_csv(r'boston_corrected.csv')
boston_gdf  = gpd.GeoDataFrame(boston_df, geometry=gpd.points_from_xy(boston_df.LAT, boston_df.LON))

hospital_df = pd.read_csv('massachusetts-hospitals.csv')
hospital_gdf  = gpd.GeoDataFrame(hospital_df, geometry=gpd.points_from_xy(hospital_df.SHAPE_Y, hospital_df.SHAPE_X))

school_df = pd.read_csv('massachusetts-schools-pre-k-through-high-school.csv')
school_gdf  = gpd.GeoDataFrame(school_df, geometry=gpd.points_from_xy(school_df.SHAPE_Y, school_df.SHAPE_X))

In [ ]:
boston_gdf.to_crs(epsg=27700, inplace=True)
hospital_gdf.to_crs(epsg=27700, inplace=True)
school_gdf.to_crs(epsg=27700, inplace=True)

print(boston_gdf.crs.axis_info[0].unit_name)

In [ ]:
boston_gdf['dist_nearest_hospital'] = [np.inf for _ in range(len(boston_gdf))]
for idx, hospital in hospital_gdf.iterrows():
    boston_gdf['dist_nearest_hospital'] = [min(a,b) for a,b in zip(boston_gdf['dist_nearest_hospital'],
                                              boston_gdf.geometry.distance(hospital.geometry))]
    
boston_gdf['dist_nearest_school'] = [np.inf for _ in range(len(boston_gdf))]
for idx, school in school_gdf.iterrows():
    boston_gdf['dist_nearest_school'] = [min(a,b) for a,b in zip(boston_gdf['dist_nearest_school'],
                                              boston_gdf.geometry.distance(school.geometry))]

In [ ]:
# Setup colors for each town 
num_town_ids = gdf['TOWNNO'].nunique()

In [ ]:
# Color vector with 92 unique colors
colors = []

for i in range(num_town_ids):
    colors.append('#%06X' % randint(0, 0xFFFFFF))

In [11]:
# # Read in the geoJSON files
# landmarks = os.path.join('Boston_Landmarks_Commission_(BLC)_Historic_Districts.geojson')
# unis = os.path.join('Colleges_and_Universities.geojson')
# com_pools = os.path.join('Community_Center_Pools.geojson')
# com_centers = os.path.join('Community_Centers.geojson')
# fire_deps = os.path.join('Fire_Departments.geojson')
# hosps = os.path.join('Hospitals.geojson')
# non_pub_schs = os.path.join('Non_Public_Schools.geojson')
# open_spaces = os.path.join('Open_Space.geojson')
# precincts = os.path.join('Precincts.geojson')
# ZIPs = os.path.join('ZIP_Codes.geojson')
# zones = os.path.join('Zoning_Subdistricts.geojson')
unis = os.path.join('University.json')

In [ ]:
# # Create a map
# m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# # Add points to the map
# for idx, row in boston_corrected.iterrows():
# #     print(boston_corrected.loc[idx,'TOWNNO'])
# #     print(colors[boston_corrected.loc[idx,'TOWNNO']])
#     Marker([row['LAT'], row['LON']],icon=folium.Icon(color='black',icon_color=colors[row['TOWNNO']])).add_to(m_2)

# # Add    
# folium.GeoJson(precincts, name = 'uni').add_to(m_2)

# # Display the map
# #m_2

In [8]:
# Create a map
m_1 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add a bubble map to the base map
for i in range(0,len(boston_df)):
    Circle(
        location=[boston_df.iloc[i]['LAT'], boston_df.iloc[i]['LON']],
        radius=50, color = 'red').add_to(m_1)
    
# Add GeoJson files
# folium.GeoJson(precincts, name = 'precincts').add_to(m_1)
# folium.GeoJson(hosps, name = 'hosps').add_to(m_1)
# folium.GeoJson(fire_deps, name = 'fire_deps').add_to(m_1)
# folium.GeoJson(non_pub_schs, name = 'non_pub_schs').add_to(m_1)
# folium.GeoJson(unis, name = 'unis').add_to(m_1)
# folium.GeoJson(com_pools, name = 'com_pools').add_to(m_1)
# folium.GeoJson(com_centers, name = 'com_centers').add_to(m_1)
folium.GeoJson(unis, name = 'com_centers').add_to(m_1)

# Display the map
m_1

In [ ]:
hospitals.head()

In [ ]:
# Create a map
m_2 = folium.Map(location=[42.32,-71.0589], tiles='cartodbpositron', zoom_start=13)

# Add a bubble map to the base map
for i in range(0,len(boston_corrected)):
    Circle(
        location=[boston_corrected.iloc[i]['LAT'], boston_corrected.iloc[i]['LON']],
        radius=100,
        color=colors[boston_corrected.iloc[i]['TOWNNO']]).add_to(m_2)
    
# Add a bubble map for hospitals
for i in range(0,len(hospitals)):
    Circle(
        location=[hospitals.iloc[i]['SHAPE_Y'], hospitals.iloc[i]['SHAPE_X']],
        radius=10,
        color='red').add_to(m_2)
    
# Add a bubble map for hospitals
for i in range(0,len(schools)):
    Circle(
        location=[schools.iloc[i]['SHAPE_Y'], schools.iloc[i]['SHAPE_X']],
        radius=10,
        color='black').add_to(m_2)   


# Display the map
m_2